In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,63611
2,Huelva,Confirmados PDIA 14 días,7491
3,Huelva,Tasa PDIA 14 días,"1459,750180252158"
4,Huelva,Confirmados PDIA 7 días,3458
5,Huelva,Tasa PDIA 7 dias,"673,8507706997681"
6,Huelva,Total Confirmados,63831
7,Huelva,Curados,50153
8,Huelva,Fallecidos,431


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  63611.0


/tmp/ipykernel_4569/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  18515.0


/tmp/ipykernel_4569/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_4569/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4569/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4569/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 121 personas en los últimos 7 días 

Un positivo PCR cada 57 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,63611.0,7491.0,3458.0,513170.0,673.850771,1459.750180,947.0
Huelva-Costa,37388.0,4323.0,1996.0,289548.0,689.350298,1493.016702,594.0
Huelva (capital),18515.0,2542.0,1190.0,143837.0,827.325375,1767.278239,336.0
Condado-Campiña,18654.0,1737.0,682.0,156231.0,436.533082,1111.815197,183.0
Sierra de Huelva-Andévalo Central,7110.0,1412.0,772.0,67391.0,1145.553561,2095.235269,166.0
Cartaya,2828.0,316.0,163.0,20083.0,811.631728,1573.470099,59.0
Aljaraque,2511.0,368.0,191.0,21474.0,889.447704,1713.700289,57.0
Lepe,3633.0,278.0,111.0,27880.0,398.134864,997.130560,44.0
Valverde del Camino,1577.0,310.0,155.0,12750.0,1215.686275,2431.372549,43.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,24.0,22.0,7.0,330.0,2121.212121,6666.666667,2.0
Rosal de la Frontera,167.0,66.0,37.0,1697.0,2180.318209,3889.216264,0.0
Cerro de Andévalo (El),308.0,69.0,48.0,2327.0,2062.741728,2965.191233,7.0
Jabugo,187.0,67.0,30.0,2260.0,1327.433628,2964.601770,3.0
Villanueva de las Cruces,43.0,11.0,2.0,387.0,516.795866,2842.377261,1.0
Valverde del Camino,1577.0,310.0,155.0,12750.0,1215.686275,2431.372549,43.0
Campillo (El),184.0,49.0,37.0,2024.0,1828.063241,2420.948617,7.0
Nerva,451.0,124.0,89.0,5169.0,1721.803057,2398.916618,27.0
Zalamea la Real,246.0,73.0,54.0,3054.0,1768.172888,2390.307793,14.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Almendro (El),77.0,8.0,1.0,840.0,119.047619,952.380952,0.0,0.125000
Villanueva de las Cruces,43.0,11.0,2.0,387.0,516.795866,2842.377261,1.0,0.181818
Lucena del Puerto,452.0,35.0,7.0,3261.0,214.658080,1073.290402,1.0,0.200000
Manzanilla,227.0,23.0,5.0,2118.0,236.071766,1085.930123,0.0,0.217391
Santa Bárbara de Casa,61.0,8.0,2.0,1043.0,191.754554,767.018217,0.0,0.250000
Alosno,345.0,34.0,9.0,3933.0,228.832952,864.480041,3.0,0.264706
Paterna del Campo,382.0,55.0,15.0,3457.0,433.902227,1590.974834,2.0,0.272727
Calañas,334.0,50.0,14.0,2768.0,505.780347,1806.358382,1.0,0.280000
Campofrío,94.0,17.0,5.0,713.0,701.262272,2384.291725,2.0,0.294118
